In [1]:
import pandas as pd
import numpy as np
import os, re
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
from collections import Counter
from datetime import datetime

%matplotlib inline

In [2]:
df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\train_df_fe_all.csv")
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,annual_TURC,annual_RURC,annual_TP,annual_AMHIFWPHMECC,quarterly_GICD,quarterly_ULCOOE,quarterly_HPI,monthly_PLR,monthly_SDR,monthly_CPI
0,2001-08,pasir ris,4 room,2,pasir ris drive 4,01 to 06,118.0,model a,uncategorized,1989,...,2.7,3.8,4138012.0,1925.0,47195.8,93.0,70.6,5.80,1.28,75.299
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,2.6,3.7,5469724.0,3418.0,105970.4,101.5,137.0,5.35,0.11,99.255
2,2020-09,sengkang,5 room,2,fernvale lane,01 to 06,112.0,premium apartment,uncategorized,2004,...,3.8,5.2,5685807.0,4022.0,113497.3,88.2,133.9,5.25,0.10,100.139
3,2000-10,clementi,3 room,0,clementi avenue 4,06 to 10,67.0,new generation,uncategorized,1980,...,4.4,6.0,4027887.0,1735.0,50041.6,99.2,75.8,5.80,1.28,74.844
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,2.6,3.9,5399162.0,3204.0,95412.4,101.0,148.6,5.38,0.11,98.121


### 'median_resale_price'

In [20]:
hdb = pd.read_excel(r"D:\29 CS5228\03 Project\HDBResalePrice\data\auxiliary-data\macro_factors.xlsx", 
                    sheet_name='median-resale-prices-for-regist', engine='openpyxl')
hdb.head()

,quarter,town,flat_type,price
0,2007-Q2,Ang Mo Kio,1-room,na
1,2007-Q2,Ang Mo Kio,2-room,-
2,2007-Q2,Ang Mo Kio,3-room,172000
3,2007-Q2,Ang Mo Kio,4-room,260000
4,2007-Q2,Ang Mo Kio,5-room,372000


In [21]:
hdb = hdb.applymap(lambda x: np.nan if x in ['-','', 'na', 'none'] else x)

In [22]:
def clean_flat_type(df):
    df['flat_type'] = df['flat_type'].apply(lambda x: str(x).lower())
    df.loc[df['flat_type'] == "1-room", 'flat_type'] = "1 room"
    df.loc[df['flat_type'] == "2-room", 'flat_type'] = "2 room"
    df.loc[df['flat_type'] == "3-room", 'flat_type'] = "3 room"
    df.loc[df['flat_type'] == "4-room", 'flat_type'] = "4 room"
    df.loc[df['flat_type'] == "5-room", 'flat_type'] = "5 room"
    return df

hdb = clean_flat_type(hdb)
hdb.head()

,quarter,town,flat_type,price
0,2007-Q2,Ang Mo Kio,1 room,NaN
1,2007-Q2,Ang Mo Kio,2 room,NaN
2,2007-Q2,Ang Mo Kio,3 room,172000.0
3,2007-Q2,Ang Mo Kio,4 room,260000.0
4,2007-Q2,Ang Mo Kio,5 room,372000.0


In [23]:
hdb = hdb.rename(columns={'quarter': 'resale_quarter', 'price': 'median_resale_price'})
hdb[['resale_year', 'resale_quarter']] = hdb['resale_quarter'].str.split('-', 1, expand=True)
hdb['resale_year'] = hdb['resale_year'].apply(lambda x: int(x))
hdb['resale_quarter'] = hdb['resale_quarter'].apply(lambda x: int(x[1]))
hdb['town'] = hdb['town'].apply(lambda x: str(x).lower())
df['town'] = df['town'].apply(lambda x: str(x).lower())
hdb.head()

,resale_quarter,town,flat_type,median_resale_price,resale_year
0,2,ang mo kio,1 room,NaN,2007
1,2,ang mo kio,2 room,NaN,2007
2,2,ang mo kio,3 room,172000.0,2007
3,2,ang mo kio,4 room,260000.0,2007
4,2,ang mo kio,5 room,372000.0,2007


In [26]:
df_x_aux = pd.merge(df, hdb, how='left')[['median_resale_price']]
df_x_aux

,median_resale_price
0,NaN
1,460900.0
2,NaN
3,NaN
4,339000.0
...,...
431727,NaN
431728,415900.0
431729,326000.0
431730,538500.0


### 'no_of_resale_applications'

In [28]:
hdb = pd.read_excel(r"D:\29 CS5228\03 Project\HDBResalePrice\data\auxiliary-data\macro_factors.xlsx", 
                    sheet_name='number-of-resale-applications-r', engine='openpyxl')
hdb = hdb.applymap(lambda x: np.nan if x in ['-','', 'na', 'none'] else x)
hdb = clean_flat_type(hdb)
hdb.head()

,quarter,flat_type,no_of_resale_applications
0,2007-Q1,1 room,5
1,2007-Q1,2 room,67
2,2007-Q1,3 room,1908
3,2007-Q1,4 room,2365
4,2007-Q1,5 room,1402


In [29]:
hdb = hdb.rename(columns={'quarter': 'resale_quarter'})
hdb[['resale_year', 'resale_quarter']] = hdb['resale_quarter'].str.split('-', 1, expand=True)
hdb['resale_year'] = hdb['resale_year'].apply(lambda x: int(x))
hdb['resale_quarter'] = hdb['resale_quarter'].apply(lambda x: int(x[1]))
hdb.head()

,resale_quarter,flat_type,no_of_resale_applications,resale_year
0,1,1 room,5,2007
1,1,2 room,67,2007
2,1,3 room,1908,2007
3,1,4 room,2365,2007
4,1,5 room,1402,2007


In [30]:
df_x_aux = pd.concat([df_x_aux, pd.merge(df, hdb, how='left')[['no_of_resale_applications']]], axis=1)
df_x_aux.head()

,median_resale_price,no_of_resale_applications
0,NaN,NaN
1,460900.0,1024.0
2,NaN,1954.0
3,NaN,NaN
4,339000.0,1362.0


### 'construction_status'

In [45]:
hdb = pd.read_excel(r"D:\29 CS5228\03 Project\HDBResalePrice\data\auxiliary-data\macro_factors.xlsx", 
                    sheet_name='completion-status-of-hdb-reside', engine='openpyxl')
hdb.head()

,financial_year,town_or_estate,hdb_or_dbss,status,no_of_units
0,2008,Ang Mo Kio,HDB,Under Construction,1221
1,2008,Ang Mo Kio,HDB,Completed,0
2,2008,Bedok,HDB,Under Construction,0
3,2008,Bedok,HDB,Completed,572
4,2008,Bishan,HDB,Under Construction,176


In [46]:
hdb = hdb.rename(columns={'financial_year': 'resale_year', 'town_or_estate': 'town'})
hdb = hdb.groupby(['resale_year', 'town', 'hdb_or_dbss', 'status'])['no_of_units'].sum().unstack(['hdb_or_dbss', 'status']).reset_index()
hdb.columns = [i[0] if i[1]=='' else i[0]+'_'+i[1] for i in hdb.columns]
hdb.head()

,resale_year,town,HDB_Completed,HDB_Under Construction,DBSS_Completed,DBSS_Under Construction
0,2008,Ang Mo Kio,0.0,1221.0,NaN,NaN
1,2008,Bedok,572.0,0.0,NaN,NaN
2,2008,Bishan,0.0,176.0,NaN,NaN
3,2008,Bukit Batok,0.0,0.0,NaN,NaN
4,2008,Bukit Merah,0.0,2718.0,NaN,NaN


In [47]:
hdb.columns

Index(['resale_year', 'town', 'HDB_Completed', 'HDB_Under Construction',
       'DBSS_Completed', 'DBSS_Under Construction'],
      dtype='object')

In [48]:
hdb['town'] = hdb['town'].apply(lambda x: str(x).lower())
df['town'] = df['town'].apply(lambda x: str(x).lower())
df_x_aux = pd.concat([df_x_aux, pd.merge(df, hdb, how='left')[
    ['HDB_Completed', 'HDB_Under Construction', 'DBSS_Completed', 'DBSS_Under Construction']]], axis=1)
df_x_aux.head()

,median_resale_price,no_of_resale_applications,HDB_Completed,HDB_Under Construction,DBSS_Completed,DBSS_Under Construction
0,NaN,NaN,NaN,NaN,NaN,NaN
1,460900.0,1024.0,4638.0,14394.0,0.0,0.0
2,NaN,1954.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,339000.0,1362.0,180.0,2886.0,0.0,0.0


### 'resale_transactions'

In [50]:
hdb = pd.read_excel(r"D:\29 CS5228\03 Project\HDBResalePrice\data\auxiliary-data\macro_factors.xlsx", 
                    sheet_name='resale-transactions-by-flat-typ', engine='openpyxl')
hdb = hdb.applymap(lambda x: np.nan if x in ['-','', 'na', 'none'] else x)
hdb = clean_flat_type(hdb)
hdb = hdb.rename(columns={'financial_year': 'resale_year'})
hdb['resale_year'] = hdb['resale_year'].apply(lambda x: int(x))
df_x_aux = pd.concat([df_x_aux, pd.merge(df, hdb, how='left')[['resale_transactions']]], axis=1)
df_x_aux.head()

,median_resale_price,no_of_resale_applications,HDB_Completed,HDB_Under Construction,DBSS_Completed,DBSS_Under Construction,resale_transactions
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,460900.0,1024.0,4638.0,14394.0,0.0,0.0,3822.0
2,NaN,1954.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,339000.0,1362.0,180.0,2886.0,0.0,0.0,5553.0


# REVIEW

In [54]:
df_x_aux.columns

Index(['median_resale_price', 'no_of_resale_applications', 'HDB_Completed',
       'HDB_Under Construction', 'DBSS_Completed', 'DBSS_Under Construction',
       'resale_transactions'],
      dtype='object')

In [51]:
review = pd.concat([df, df_x_aux], axis=1)
review.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,monthly_PLR,monthly_SDR,monthly_CPI,median_resale_price,no_of_resale_applications,HDB_Completed,HDB_Under Construction,DBSS_Completed,DBSS_Under Construction,resale_transactions
0,2001-08,pasir ris,4 room,2,pasir ris drive 4,01 to 06,118.0,model a,uncategorized,1989,...,5.80,1.28,75.299,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,5.35,0.11,99.255,460900.0,1024.0,4638.0,14394.0,0.0,0.0,3822.0
2,2020-09,sengkang,5 room,2,fernvale lane,01 to 06,112.0,premium apartment,uncategorized,2004,...,5.25,0.10,100.139,NaN,1954.0,NaN,NaN,NaN,NaN,NaN
3,2000-10,clementi,3 room,0,clementi avenue 4,06 to 10,67.0,new generation,uncategorized,1980,...,5.80,1.28,74.844,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,5.38,0.11,98.121,339000.0,1362.0,180.0,2886.0,0.0,0.0,5553.0


In [53]:
review[(df['resale_year']>=2008) & (df['resale_year']<=2019)]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,monthly_PLR,monthly_SDR,monthly_CPI,median_resale_price,no_of_resale_applications,HDB_Completed,HDB_Under Construction,DBSS_Completed,DBSS_Under Construction,resale_transactions
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,5.35,0.11,99.255,460900.0,1024.0,4638.0,14394.0,0.0,0.0,3822.0
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,5.38,0.11,98.121,339000.0,1362.0,180.0,2886.0,0.0,0.0,5553.0
7,2010-11,geylang,5 room,0,pine close,16 to 21,110.0,improved,uncategorized,2000,...,5.38,0.13,88.818,NaN,1477.0,354.0,0.0,NaN,NaN,7274.0
9,2013-10,bukit batok,3 room,1,bukit batok street 52,10 to 15,60.0,improved,uncategorized,1988,...,5.38,0.10,99.020,326000.0,1251.0,180.0,2886.0,0.0,0.0,5553.0
10,2010-06,bukit panjang,executive,1,jelapang road,01 to 06,140.0,apartment,uncategorized,1999,...,5.38,0.14,86.567,NaN,768.0,456.0,1996.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431724,2019-05,queenstown,5 room,0,dover crescent,21 to 25,125.0,standard,uncategorized,1976,...,5.25,0.16,100.325,NaN,1520.0,489.0,3219.0,0.0,0.0,6005.0
431726,2011-04,jurong west,3 room,0,boon lay place,10 to 15,65.0,improved,uncategorized,1976,...,5.38,0.12,90.774,286000.0,1966.0,0.0,2533.0,0.0,682.0,7230.0
431728,2016-04,sengkang,4 room,1,fernvale road,10 to 15,95.0,premium apartment,uncategorized,2012,...,5.35,0.14,98.365,415900.0,2473.0,2934.0,3216.0,0.0,0.0,8651.0
431729,2011-01,tampines,3 room,0,tampines street 81,01 to 06,67.0,new generation,uncategorized,1986,...,5.38,0.12,90.405,326000.0,1818.0,384.0,2670.0,0.0,708.0,7230.0
